In [1]:
import time
    v
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn import metrics

import os
import joblib

In [2]:
cici_test_dir='/home/irteam/dcloud-global-dir/MLAC/new_data/test'
X_test=pd.read_csv(os.path.join(cici_test_dir,'CICI_Xtest.csv'))
y_test=pd.read_csv(os.path.join(cici_test_dir,'CICI_ytest.csv'))

In [3]:
df=pd.DataFrame(columns=['L1','b_acc','b_f1','b_rc','b_pc','L2','m_acc','m_f1','m_rc','m_pc']+\
                 ['C1','c1_acc','c1_f1','c1_rc','c1_pc']+\
                    ['C2','c2_acc','c2_f1','c2_rc','c2_pc','C3','c3_acc','c3_f1','c3_rc','c3_pc','C4','c4_acc','c4_f1','c4_rc','c4_pc']+\
                     ['final_acc','final_f1','final_rc','final_pc'])

outpath='/home/irteam/junghye-dcloud-dir/MLAC/230624/CICI'
cnt=0
model_eval=[]

In [5]:
X_test.head()

,protocol,flow_duration,tot_fwd_pkts,tot_bwd_pkts,tot_len_fwd_pkts,tot_len_bwd_pkts,fwd_pkt_len_max,fwd_pkt_len_min,fwd_pkt_len_mean,fwd_pkt_len_std,...,init_bwd_win_byts,fwd_act_data_pkts,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min
0,0.385385,0.279279,0.000000,0.370370,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.901902,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.385385,0.528550,0.856356,0.871371,0.751251,0.897898,0.694194,0.000000,0.524024,0.767267,...,0.730230,0.888388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.385385,0.331832,0.482482,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.385385,0.443333,0.738739,0.700200,0.958580,0.779005,0.990885,0.000000,0.970421,0.974667,...,0.517518,0.640641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.000000,0.725435,0.311311,0.136136,0.470971,0.600045,0.578579,0.894895,0.720721,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
L1_ytest=y_test.copy()

L2_ytest=X_test['nist_category'].copy()

L3_ytest=X_test['attack_category'].copy()

c1_Xtest=X_test.query('nist_category==1')
c1_ytest=c1_Xtest['attack_category']

c2_Xtest=X_test.query('nist_category==2')
c2_ytest=c2_Xtest['attack_category']

c3_Xtest=X_test.query('nist_category==3')
c3_ytest=c3_Xtest['attack_category']

c4_Xtest=X_test.query('nist_category==4')
c4_ytest=c4_Xtest['attack_category']

KeyError: 'nist_category'

In [ ]:
for class_data in [c1_Xtest,c2_Xtest,c3_Xtest,c4_Xtest]:
    class_data.drop(labels=['attack_category','nist_category'],axis=1,inplace=True)

In [ ]:
X_test.drop(labels=['nist_category','attack_category'],axis=1,inplace=True)

In [ ]:

def test_result(model:str,test,pred) ->list:

    acc=accuracy_score(test,pred)
    f1=f1_score(test,pred,average='weighted')
    recall=recall_score(test,pred,average='weighted')
    precision=precision_score(test,pred,average='weighted')

    print(f'{model} result , acc:{acc}, f1:{f1},recall:{recall},precision:{precision}')
    return([acc,f1,recall,precision])


In [ ]:
saved_path='/home/irteam/dcloud-global-dir/MLAC/saved_models/230620/savedmodels'


In [ ]:
L1_model=joblib.load(os.path.join(saved_path,'CICI_L1mod_rc.pkl'))



In [ ]:
L1_ypred=L1_model.predict(X_test)

eval_result=test_result(L1_model,L1_ytest,L1_ypred)

model_eval.append('L1')

model_eval.extend(eval_result)

In [ ]:
malicious_indices=np.where(L1_ypred==1)[0]
model_eval.append('L2')

if malicious_indices.any():
    L2_model=joblib.load(os.path.join(saved_path,'CICI_nist.pkl'))
    L2_Xtest=X_test.iloc[malicious_indices]
    L2_ypred=L2_model.predict(L2_Xtest)
    L2_ytest_selected=L2_ytest.iloc[malicious_indices]
    L2_result=test_result(L2_model,L2_ytest_selected,L2_ypred)
    
    model_eval.extend(L2_result)
else:
    print('no malicious predicted')
    import sys
    sys.exit()
    

In [ ]:
c1_model=joblib.load(os.path.join(saved_path,'class_1_CICI.pkl'))
model_eval.append('c1')
indices1=np.where(L2_ypred==1)[0]

print('Reconnaissance train & test')

if indices1.any():
    c1_Xtest_selected=L2_Xtest.iloc[indices1]
    c1_ypred=c1_model.predict(c1_Xtest_selected)
    c1_ytest_selected=L3_ytest.iloc[indices1]
    c1_result=test_result(c1_model,c1_ytest_selected,c1_ypred)
    model_eval.extend(c1_result)
else:
    model_eval.extend([0,0,0,0])

c2_model=joblib.load(os.path.join(saved_path,'class_2_CICI.pkl'))
model_eval.append('c2')
indices2=np.where(L2_ypred==2)[0]

print('Access train & test ')

if indices2.any():
    c2_Xtest_selected=L2_Xtest.iloc[indices2]
    c2_ypred=c2_model.predict(c2_Xtest_selected)
    c2_ytest_selected=L3_ytest.iloc[indices2]
    c2_result=test_result(c2_model,c2_ytest_selected,c2_ypred)
    
    model_eval.extend(c2_result)

else:
    model_eval.extend([0,0,0,0])

c3_model=joblib.load(os.path.join(saved_path,'class_3_CICI.pkl'))
model_eval.append('c3')
indices3=np.where(L2_ypred==3)[0]
print('Dos train & test')

if indices3.any():
    c3_Xtest_selected=L2_Xtest.iloc[indices3]
    c3_ypred=c3_model.predict(c3_Xtest_selected)
    c3_ytest_selected=L3_ytest.iloc[indices3]
    c3_result=test_result(c3_model,c3_ytest_selected,c3_ypred)

    
    model_eval.extend(c3_result)

else:
    model_eval.extend([0,0,0,0])

c4_model=joblib.load(os.path.join(saved_path,'class_4_CICI.pkl'))
model_eval.append('c4')
indices4=np.where(L2_ypred==4)[0]
print('Malware train & test')
if indices4.any():
    c4_Xtest_selected=L2_Xtest.iloc[indices4]
    c4_ypred=c4_model.predict(c4_Xtest_selected)
    c4_ytest_selected=L3_ytest.iloc[indices4]
    c4_result=test_result(c4_model,c4_ytest_selected,c4_ypred)
    
    model_eval.extend(c4_result)

else:
    model_eval.extend([0,0,0,0])






In [ ]:
df.loc[cnt]=model_eval
df.to_csv(os.path.join(outpath,'CICI_result.csv'))


In [ ]:
df